In [ ]:
! pip install transformers

In [ ]:
# install Spacy and a language model

!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm


# IF YOU ARE USING A NEW APPLE (M1 CPU) COMPUTER use these lines instead.
# !pip install -U pip setuptools wheel
# !pip install -U 'spacy[apple]'
# !python -m spacy download en_core_web_sm

# SC207 Text Mining
## Sentiment Analysis and Entity Recognition
### Using Pre-Trained Models for quick text insights

The two methods we're using today rely on pre-trained models to quickly pull apart and analyse pieces of text with a high degree of complexity. Trained on millions of examples of text from the internet, archives, books etc. These models go beyond simply looking at what words are being used, and consider the placement of words, their immediate and distant context, their role within sentence structure and more, to make inferences about what the text says, what matters in the text, and what it could imply.

#### Tools
Today we're using two packages.
- [Transformers](https://pypi.org/project/transformers/): Allows us to quickly download a pre-trained model designed specifically for sentiment analysis using the HuggingFace 🤗 [AI model repository](https://huggingface.co/)
- [SpaCy](https://spacy.io/): A natural language processing package that relies on its own pre-trained models to provide a large set of text analysis features. Today we'll be using itrs powerful entity recognition system.

In [ ]:
import pandas as pd
from transformers import pipeline
import spacy
import matplotlib.pyplot as plt
import seaborn as sns

# Sentiment Analysis

A tricky area to get right. Prior to pre-trained models sentiment was determined by matching specific words to a predefined table that gave each word a score depending on how positive/negative the designers felt the word was. Whilst this worked for simple text, sentiment is often context dependent, can be morphed by sarcasm, and changes over time as lagnuage evolves. Regularly updated models are shown examples of text that have been labelled as either positive or negative by human annotators, and tested to see if they can accurately predict what a human would label a brand new piece of text.

We can initialise one of those super complex incredibly difficult to build models, but it will take every bit of our coding skills to do so...

In [ ]:
get_sentiment = pipeline('sentiment-analysis')

Done.

In [ ]:
get_sentiment("I love every brilliant thing right now. Super happy")

In [ ]:
get_sentiment("My name is James")

In [ ]:
get_sentiment("I am very angry")

In [ ]:
get_sentiment("""A New York judge has ordered President Donald Trump to pay $2m (£1.6m)"""
              """ for misusing funds from his charity to finance his 2016 political campaign."""
              """ The Donald J Trump Foundation closed down in 2018. Prosecutors had accused it"""
              """ of working as "little more than a chequebook" for Mr Trump's interests."""
              """ Charities such as the one Mr Trump and his three eldest children headed cannot"""
              """ engage in politics, the judge ruled.""")

It is worth noting three things...
1. Text can ONLY be positive or negative under this model, there is no neutral.
2. The score does not indicate strength of sentiment. It indicates how confident the model is in its prediction. We'll look at this more later.
3. Always consider does it make sense for a text to have a sentiment. The model will always assign one but this doesn't mean it makes sense.

## Reshaping

We will use our twitter dataset and also our community assignments we generated using NetworkX. We can use these later to examine whether sentiment differs between different groups in our retweet network.

In [ ]:
tweets = pd.read_json('trhr.json')
tweets.info()

In [ ]:
communities = pd.read_csv('communities.csv', index_col=0)
communities.head()

In [ ]:
# Merging together tweet data and community assignments
tweets = tweets.merge(communities,how='left', left_on='user_id', right_index=True).dropna(subset=['community'])
tweets.shape

In [ ]:
sample = tweets.sample(500).copy().reset_index()
sample = sample[['created_at','text','community']]

In [ ]:
sample['sentiment'] = get_sentiment(sample['text'].tolist())
sample

In [ ]:
label_score = pd.json_normalize(sample['sentiment'])
sample = pd.concat([sample,label_score], axis=1)

sample['community'] = sample['community'].astype(int)

sample.head()

## Visuals 
### Distribution of Sentiment

Basic Frequency of Positive or Negative Label

In [ ]:
sample.groupby('label').count()

In [ ]:
sns.catplot(data=sample, x='label', kind='count')

In [ ]:
pd.crosstab(sample['community'], sample['label'])

Frequency split by community

In [ ]:
pd.crosstab(sample['community'], sample['label'])

In [ ]:
sns.catplot(data=sample, y='community',hue='label', kind='count')

In [ ]:
top_5_communities = sample['community'].value_counts().head().index
top_5_communities

In [ ]:
sample = sample[sample['community'].isin(top_5_communities)].copy() # We use .copy to make it a new object rather than a view as we'll probably change this df later

In [ ]:

sns.catplot(data=sample, y='community',hue='label', kind='count',order=top_5_communities).set(title='Sentiment of Tweets ordered by community tweet freq')


### Score Confidence
The confidence score indicates how confident the model is in making its prediction. Whilst individual scores are not an indicator of strength of sentiment, we can see if communities are expressing themselves in ways that are obviously positive or negative. Instances where communities may not have high scores do not indicate neutrality, simply that the language is not clearly positive or negative *to the model*.

In [ ]:
# Grouping first by community and running describe on the score column gives us a range of stats per community
# Note some communities are only one person large
sample.groupby('community')['score'].describe()

In [ ]:
sns.catplot(data=sample, x='community', y='score',kind='box', hue='label', aspect=2,order=top_5_communities)

Average confidence scores per community and label.

In [ ]:
confidence_data = sample.groupby(['community','label']).mean().unstack()
confidence_data

In [ ]:
sns.heatmap(data=confidence_data, annot=True, linewidths=0.3,  cmap='coolwarm',vmin=0.5, vmax=1)

## Sentiment over time
Can we examine changing sentiment over time? We'll start with the most direct approach to visualising this, and improve where we can.

First we start with basic frequencies per hour for each label type.

In [ ]:
time_grouper = pd.Grouper(key='created_at',freq='h')
time_series_sentiment = sample.groupby([time_grouper,'label']).count().reset_index()
time_series_sentiment

In [ ]:
sns.relplot(data=time_series_sentiment,x='created_at', y='score', hue='label', kind='line')

How about improving this so it is one line, that represents the average sentiment, if we imagine that more positively labelled tweets pulls the score up, and negative labelled ones pull the score down.

In [ ]:
sample['positivity'] = sample['label'].map({'POSITIVE':1, 'NEGATIVE':0})
sample

In [ ]:
avg_positivity = sample.groupby(time_grouper).mean().reset_index()

In [ ]:
sns.relplot(data=avg_positivity,x='created_at', y='positivity', kind='line')

As we have low sample rates for some of our hours the score swings wildly from 0 to 1 and back again. We can smooth this into a trend line using the `.rolling` method of pandas.

In [ ]:
avg_positivity['smoothed'] = avg_positivity['positivity'].rolling(3).mean()
avg_positivity

In [ ]:
sns.relplot(data=avg_positivity, x='created_at',y='smoothed', kind='line').set(ylim=(0,1))
# we can use .set to ensure the y axis runs from 0 to 1 to tell the full story.

# We can also slightly rotate the 'tick' labels on the x-axis to make them more readable.
plt.xticks(rotation=45)
plt.show()

In [ ]:
time_freq = '10min'
window_size = 6

time_grouper = pd.Grouper(key='created_at', freq=time_freq)
avg_positivity = sample.groupby(time_grouper).mean().reset_index()
avg_positivity['smoothed'] = avg_positivity['positivity'].rolling(window_size).mean()

sns.relplot(data=avg_positivity, x='created_at',y='smoothed', kind='line').set(
    title=f'Rolling Average Positivity: sample_rate={time_freq} | window_size={window_size}', ylim=(0,1), xlabel='Tweet Created At', ylabel='Mean Positivity')

plt.xticks(rotation=45)
plt.show()

# Named Entity Recognition

Named entity recognition (NER) is the technique of extracting key entities within a piece of text,
- people
- places
- organisations
- dates
- values
- currencies etc.

SpaCy's processing examines each word in context and uses this to predict which tokens likely refer to particular types of entities like people, organisations, dates etc. It is not using any limited list or reference to "look up" these entities, but instead identifies them based on contextual cues.


In [ ]:
text_data = pd.read_csv('sample_news_large.csv')

In [ ]:
text_data.head()

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
text = nlp("""A New York judge has ordered President Donald Trump to pay $2m (£1.6m)"""\
            """ for misusing funds from his charity to finance his 2016 political campaign."""\
            """ The Donald J Trump Foundation closed down in 2018. Prosecutors had accused it"""\
            """ of working as "little more than a chequebook" for Mr Trump's interests."""\
            """ Charities such as the one Mr Trump and his three eldest children headed cannot"""\
            """ engage in politics, the judge ruled.""")

# Source: https://www.bbc.co.uk/news/world-us-canada-50338231

In [ ]:
# we can access the entities with the .ents attribute
text.ents

In [ ]:
# every object in the entities list has a text attribute and a label attribute to tell you the type of entity it is.

for entity in text.ents:
    print(entity.text, entity.label_)

In [ ]:
# as we're in Jupyter we can also use SpaCy's built in visualiser

spacy.displacy.render(text,style='ent', jupyter=True)

In [ ]:
# if you want to save the annotated version of the
# text you can save to html using this function.

def save_displacy_to_html(doc, filename, style='ent'):
    html_data = spacy.displacy.render(doc, style='ent', jupyter=False, page=True)
    with open(filename, 'w+', encoding="utf-8") as f:
        f.write(html_data)

save_displacy_to_html(text, 'test.html', style='ent')

In [ ]:
# lets create a function that can extract specific types of entities from a text

def entity_extractor(nlp_doc, entity_type=None, allow_duplicates=False):
    if entity_type is None:
        ents = [(ent.text,ent.label_) for ent in nlp_doc.ents]
    else:
        ents = [ent.text for ent in nlp_doc.ents if ent.label_ == entity_type.upper()]
    if not allow_duplicates:
        ents = list(set(ents))
    return ents

In [ ]:
entity_extractor(text)

In [ ]:
entity_extractor(text, 'person')

In [ ]:
docs = nlp.pipe(text_data['text'])
people = [entity_extractor(doc,'person') for doc in docs]

In [ ]:
text_data['people'] = people
text_data['people']

In [ ]:
people_data = text_data.explode('people')[['query','people','title']]
people_data


In [ ]:
# most mentioned people
people_data['people'].value_counts()[:10]

In [ ]:
# top ten people per group
for query,data in people_data.groupby('query'):
    print(f"****{query}****")
    print(data['people'].value_counts()[:10])
    print()

In [ ]:
top_people = people_data.groupby('people',as_index=False).count().nlargest(5,'query')
top_people


In [ ]:
sns.catplot(data=top_people, y='people',x='query', kind='bar',height=5, aspect=2).set(xlabel='Freq', ylabel='Person', title='5 Most Mentioned People')

In [ ]:

for query,data in people_data.groupby('query'):
    top_for_query = data.groupby('people', as_index=False).count().nlargest(5,'title')
    sns.catplot(data=top_for_query,x='title',y='people', kind='bar', aspect=2).set(title=f'{query.title()}: Top 5 People',
                                                                         xlabel='freq',
                                                                         ylabel='Person')

## Thinking of Entities Relationally
One of the skills we developed was to use networkx to start thinking relationally about data, and how we can use measures and metrics without necessarily relying purely on visualisation (though visualisation helps!).

We can intersect text analysis with network analysis by mapping the co-occurence of different people within news stories, implying that those people whose names co-occur often are probably connected in some way.

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite

import netwulf as nw

In [ ]:
def filter_by_degree(G, minimum_degree):
    scores = G.degree()
    to_keep = [node for node,degree in scores if degree >= minimum_degree]
    return G.subgraph(to_keep)

def weighted_degree_centrality(G, weight_label='weight'):
    degree_scores = G.degree(weight=weight_label)
    return {name:score for name,score in degree_scores}

# Sets the size attribute of our graph to whatever scores are passed in
def size_by(G,scores):
    nx.set_node_attributes(G,scores, name='size')
    return G

In [ ]:
edge_list = people_data.dropna()[['people']].copy()
edge_list

In [ ]:
edge_list = people_data[['people']].copy()
edge_list

In [ ]:
edge_list.rename(columns={'people':'source'}, inplace=True)
edge_list['target'] = edge_list.index
edge_list.reset_index(drop=True, inplace=True)
edge_list

In [ ]:
G = nx.from_pandas_edgelist(edge_list)

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_nodes()

In [ ]:
G.edges

### Bipartite?
<img src='https://github.com/Minyall/sc207_2022-23/blob/master/images/bipartite_projection.png?raw=true' align="right" height="200">

A bi-partite graph is one where two sets of nodes only ever form edges with nodes outside their set. So in our case, there are only edges between article nodes, and person nodes, never article to article, nor person to person.

However we want a graph that is person to person, but that somehow retains the information on how often they co-occur that comes from their common connection to article nodes. The solution is to create a 'projection' of the original graph that connects up those common edges and weights them based on how many original connections there were.

In [ ]:
#First we choose which nodes we want to 'keep' after the projection
keep_nodes = edge_list['source'].unique().tolist()
keep_nodes

In [ ]:
# Now we get our projected Graph

people_G = bipartite.weighted_projected_graph(G,keep_nodes)

In [ ]:
people_G.edges(data=True)

In [ ]:
# Now we find all edges where there was only 1 co-occurence and drop them

drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
people_G.remove_edges_from(drop_edges)

In [ ]:
people_G.number_of_nodes()

In [ ]:
# Now we remove any noes that have less than 1 degree, i.e. no connections at all.

people_G = filter_by_degree(people_G,minimum_degree=1)
people_G.number_of_nodes()

In [ ]:
# Here we create a degree score that accounts for the edge weights of any edges connected to a node
# we use our size-by function to quickly change the sizing before visualisation
degree_centrality = nx.degree_centrality(people_G)
people_G = size_by(people_G, degree_centrality)

In [ ]:
# People who co-occur with the highest number unique of people
pd.Series(degree_centrality).sort_values(ascending=False)

In [ ]:
# Highest co-occuring pairs
nx.to_pandas_edgelist(people_G).sort_values(by='weight', ascending=False)

In [ ]:
# The people most likely to be at the centrwe of the stories, they relationally connect together others.
betweenness_scores = nx.betweenness_centrality(people_G)
pd.Series(betweenness_scores).sort_values(ascending=False)

In [ ]:
# Size by the betweeness centrality and see if we're right with a visual
people_G = size_by(people_G,betweenness_scores)

stylized_network, config = nw.visualize(people_G)

In [ ]:
nw.draw_netwulf(stylized_network, figsize=10)

In [ ]:
G.edges

### Bipartite?
<img src='https://github.com/Minyall/sc207_2022-23/blob/master/images/bipartite_projection.png?raw=true' align="right" height="200">

A bi-partite graph is one where two sets of nodes only ever form edges with nodes outside their set. So in our case, there are only edges between article nodes, and person nodes, never article to article, nor person to person.

However we want a graph that is person to person, but that somehow retains the information on how often they co-occur that comes from their common connection to article nodes. The solution is to create a 'projection' of the original graph that connects up those common edges and weights them based on how many original connections there were.

In [ ]:
#First we choose which nodes we want to 'keep' after the projection
keep_nodes = edge_list['source'].unique().tolist()
keep_nodes

In [ ]:
# Now we get our projected Graph

people_G = bipartite.weighted_projected_graph(G,keep_nodes)

In [ ]:
people_G.edges(data=True)

In [ ]:
# Now we find all edges where there was only 1 co-occurence and drop them

drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
people_G.remove_edges_from(drop_edges)

In [ ]:
people_G.number_of_nodes()

In [ ]:
# Now we remove any noes that have less than 1 degree, i.e. no connections at all.

people_G = filter_by_degree(people_G,minimum_degree=1)
people_G.number_of_nodes()

In [ ]:
# Here we create a degree score that accounts for the edge weights of any edges connected to a node
# we use our size-by function to quickly change the sizing before visualisation
degree_centrality = nx.degree_centrality(people_G)
people_G = size_by(people_G, degree_centrality)

In [ ]:
# People who co-occur with the highest number unique of people
pd.Series(degree_centrality).sort_values(ascending=False)

In [ ]:
# Highest co-occuring pairs
nx.to_pandas_edgelist(people_G).sort_values(by='weight', ascending=False)

In [ ]:
# The people most likely to be at the centrwe of the stories, they relationally connect together others.
betweenness_scores = nx.betweenness_centrality(people_G)
pd.Series(betweenness_scores).sort_values(ascending=False)

In [ ]:
# Size by the betweeness centrality and see if we're right with a visual
people_G = size_by(people_G,betweenness_scores)

stylized_network, config = nw.visualize(people_G)

In [ ]:
nw.draw_netwulf(stylized_network, figsize=10)

### Bipartite?
<img src='https://github.com/Minyall/sc207_2022-23/blob/master/images/bipartite_projection.png?raw=true' align="right" height="200">

A bi-partite graph is one where two sets of nodes only ever form edges with nodes outside their set. So in our case, there are only edges between article nodes, and person nodes, never article to article, nor person to person.

However we want a graph that is person to person, but that somehow retains the information on how often they co-occur that comes from their common connection to article nodes. The solution is to create a 'projection' of the original graph that connects up those common edges and weights them based on how many original connections there were.

In [ ]:
#First we choose which nodes we want to 'keep' after the projection
keep_nodes = edge_list['source'].unique().tolist()
keep_nodes

In [ ]:
# Now we get our projected Graph

people_G = bipartite.weighted_projected_graph(G,keep_nodes)

In [ ]:
people_G.edges(data=True)

In [ ]:
# Now we find all edges where there was only 1 co-occurence and drop them

drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
drop_edges = [(source,target) for source, target, attributes in people_G.edges(data=True) if attributes['weight'] == 1]
drop_edges

In [ ]:
people_G.remove_edges_from(drop_edges)

In [ ]:
people_G.number_of_nodes()

In [ ]:
# Now we remove any noes that have less than 1 degree, i.e. no connections at all.

people_G = filter_by_degree(people_G,minimum_degree=1)
people_G.number_of_nodes()

In [ ]:
# Here we create a degree score that accounts for the edge weights of any edges connected to a node
# we use our size-by function to quickly change the sizing before visualisation
degree_centrality = nx.degree_centrality(people_G)
people_G = size_by(people_G, degree_centrality)

In [ ]:
# People who co-occur with the highest number unique of people
pd.Series(degree_centrality).sort_values(ascending=False)

In [ ]:
# Highest co-occuring pairs
nx.to_pandas_edgelist(people_G).sort_values(by='weight', ascending=False)

In [ ]:
# The people most likely to be at the centrwe of the stories, they relationally connect together others.
betweenness_scores = nx.betweenness_centrality(people_G)
pd.Series(betweenness_scores).sort_values(ascending=False)

In [ ]:
# Size by the betweeness centrality and see if we're right with a visual
people_G = size_by(people_G,betweenness_scores)

stylized_network, config = nw.visualize(people_G)

In [ ]:
nw.draw_netwulf(stylized_network, figsize=10)